In [2]:
# !pip install transformers datasets

In [23]:
!pip install tqdm torch

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrases/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer
from transformers import AutoModel


model_checkpoint = "DeepPavlov/distilrubert-tiny-cased-conversational-v1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModel.from_pretrained(model_checkpoint)


In [3]:
import re
from tqdm import tqdm

In [4]:
class Dataset:
    
    def __init__(self) -> None:
        self.ids = []
        self.contexts = []
        self.questions = []
        self.answers = []
        self.spans = []
ds = Dataset()

In [5]:
id = 0
contexts = raw_datasets['train']['context'] + raw_datasets['validation']['context']
questions = raw_datasets['train']['context'] + raw_datasets['validation']['question']
answers = raw_datasets['train']['context'] + raw_datasets['validation']['answers']
for i in tqdm(range(len(contexts))):

    context = contexts[i]
    answer = answers[i]
    try:
        r =  re.search(answer, context)
    
        s, e = r.span()
        ds.ids.append(id)
        id += 1
        ds.contexts.append(context)
        ds.questions.append(questions[i])
        ds.answers.append(answer)
        ds.spans.append({'start': s, 'end': e})
    except:
        None


100%|██████████| 98169/98169 [00:15<00:00, 6328.00it/s]


In [6]:
from collections import defaultdict
import torch
hidden_dim = 264
treshold = 100

In [23]:
token_id2hiddens = defaultdict(list)
for context in tqdm(ds.contexts[:treshold]):
    input_ids = tokenizer(context, truncation=True, max_length=512, return_tensors="pt")
    last_hidden_state = model(**input_ids).last_hidden_state
    for token_num, token_id in enumerate(input_ids['input_ids'][0]):
        token_id2hiddens[token_id.item()].append(last_hidden_state[0][token_num])


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:03<00:00, 31.89it/s]


In [29]:
token_id2hidden = {} 
for token_id, hiddens in token_id2hiddens.items():
    token_id2hidden[(token_id)] = torch.mean(torch.cat(hiddens).reshape(
        (len(hiddens), hidden_dim)), axis=0)

In [30]:
(token_id2hidden.keys())

dict_keys([2, 24518, 6347, 87, 4863, 25424, 6013, 21124, 1125, 10594, 20, 17065, 10951, 230, 118, 4891, 157, 71, 3053, 24775, 12681, 257, 5227, 252, 49, 66, 29069, 2498, 17473, 24488, 3639, 17540, 21650, 24, 3999, 29290, 45, 2624, 18473, 3414, 11426, 10180, 2730, 4465, 111, 16883, 20670, 6649, 3293, 21175, 3738, 28824, 4413, 22222, 5313, 986, 24228, 29725, 1704, 155, 12429, 25140, 1071, 7061, 13488, 96, 5339, 20502, 10529, 25028, 1982, 7463, 1902, 25963, 1661, 5378, 1337, 5243, 9050, 262, 23361, 10649, 14243, 1651, 3147, 13892, 17007, 21392, 2824, 5162, 16263, 6058, 16598, 775, 256, 22863, 10742, 21490, 17128, 2686, 21174, 9418, 21540, 3406, 22599, 158, 7263, 16664, 652, 7384, 14431, 23818, 100, 6508, 13227, 6078, 26438, 22498, 29302, 300, 12663, 25279, 17760, 17771, 260, 27833, 4930, 14911, 13008, 108, 7937, 1931, 26053, 9049, 135, 24226, 18384, 76, 7031, 290, 11149, 1085, 19952, 6442, 9874, 286, 1747, 9688, 8215, 109, 237, 11285, 25185, 19128, 23409, 3902, 6764, 2729, 8692, 9870, 170